In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests as req
import urllib
import time
from datetime import datetime

In [2]:
def tranform_data(sheet):
    data_ddf = pd.read_excel("treatment and control for PFE.xlsx",sheet_name=sheet,dtype={"symb":np.str})
    data_ddf["l1"]="https://finance.yahoo.com/quote/"
    data_ddf["l2"]="/history?period1=1581984000&period2=1584489600&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true"
    data_ddf["link"]= data_ddf["l1"].str.cat(data_ddf.loc[:,["symb","l2"]])
    data_ddf["Ets"]=data_ddf["Ets"].str.strip().str.capitalize()
    return data_ddf

In [3]:
def scrap_data(data_ddf):
    data = []
    data_div=[] #For dividends
    data_unique_ddf= data_ddf.drop_duplicates(subset=["Ets","symb"])
    for _,row in pd.DataFrame.iterrows(data_unique_ddf):
        #print(row.Ets,row.link)
        link = row.link
        headers = {
            "User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:97.0) Gecko/20100101 Firefox/97.0"
        }
        response = req.get(link,headers=headers)
        soup=BeautifulSoup(response.content,"html.parser")
        tr_list = soup.find("tbody").find_all("tr")
        print(row.symb,len(tr_list))
        if len(tr_list) !=0:
            for line in tr_list:
                all_span = line.find_all("span")
                if all_span[-1].get_text()=="Dividend" :
                    data_div.append([all_span[0].text,line.find("strong").text,row.Ets])
                else:
                    list_values = list(map(lambda x: datetime.strptime(x.get_text(),"%b %d, %Y").strftime("%Y-%m-%d")  if all_span.index(x)==0 else  float(x.get_text().replace(",",""))  ,all_span))
                    if len(list_values)!=7 :
                        list_values.append(np.nan)
                        print(list_values)
                    list_values.append(row.Ets)
                    list_values.append(len(tr_list))
                    list_values.append(row.symb)
                    data.append(list_values)
    return data,data_div

In [5]:
sheets = ["treatment","control"]
for sheet in sheets :
    data_ddf = tranform_data(sheet)
    data,data_div = scrap_data(data_ddf)
    bourse_ddf = pd.DataFrame(data,columns=["date","open","high","low","close","adj_close","volume","ets","number_date","symb"])
    bourse_ddf.to_csv(sheet+".csv",index=False)
    pd.DataFrame(data_div,columns=["date","dividend","ets"]).to_csv(sheet+"_div"+".csv")

<ipython-input-2-2a5ef961f6fe>:2: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  data_ddf = pd.read_excel("treatment and control for PFE.xlsx",sheet_name=sheet,dtype={"symb":np.str})


jagx 21
TTM 21
LNVGY 21
066570.KS 22
['2020-03-12', 59000.0, 59000.0, 59000.0, 59000.0, 58071.42, nan]
['2020-03-09', 60300.0, 60300.0, 60300.0, 60300.0, 59350.96, nan]
msft 22
NKE 22
NTDOY 21
VFC 22
ANF 22
2353.TW 20
ADDYY 21
ALO.PA 21
AMZN 21
AAPL 21
2357.TW 20
BMW.DE 21
BDRBF 21
BOSCHLTD.NS 19
BYDDF 21
PVH 21
CRI 22
TRIN 0
000625.SZ 21
CSCO 21
TTDKY 21
1766.HK 21
DELL 21
ELUXY 21
2238.HK 21
GELYF 21
GM 22
GOOG 21
002241.SZ 21
HTHIY 21
HPQ 22
2498.TW 20
002502.SZ 21
002230.SZ 21
600100.SS 21
PCRFY 21
RL 21
005930.KS 22
['2020-03-12', 52100.0, 52100.0, 52100.0, 52100.0, 48913.69, nan]
['2020-03-09', 56500.0, 56500.0, 56500.0, 56500.0, 53044.6, nan]
SIEGY 21
SKX 21
SONY 21
PVH 21
FRCOY 21
VSCO 0
IDEXF 21
['2020-03-11', 27.53, 27.53, 27.53, 27.53, 26.49, nan]
['2020-03-06', 30.8, 30.8, 30.8, 30.8, 29.64, nan]
['2020-02-27', 30.9, 30.9, 30.9, 30.9, 29.74, nan]
['2020-02-25', 32.21, 32.21, 32.21, 32.21, 31.0, nan]
002024.SZ 21
ZGN 0
ZEEL.NS 19
SHCAY 21
VWAGY 21
PUM.DE 21
ANGL.TA 19
['2020

In [6]:
data_ddf = pd.read_csv("treatment.csv")
data_ddf.head()

,date,open,high,low,close,adj_close,volume,ets,number_date,symb
0,2020-03-17,1.323,1.323,1.200,1.224,1.224,31100.0,Jaguar,21,jagx
1,2020-03-16,1.200,1.260,1.140,1.215,1.215,53900.0,Jaguar,21,jagx
2,2020-03-13,1.275,1.338,1.134,1.200,1.200,62233.0,Jaguar,21,jagx
3,2020-03-12,1.200,1.260,1.098,1.140,1.140,85567.0,Jaguar,21,jagx
4,2020-03-11,1.515,1.530,1.269,1.350,1.350,82433.0,Jaguar,21,jagx


In [7]:
data_ddf = pd.read_csv("control.csv")
data_ddf.head()

,date,open,high,low,close,adj_close,volume,ets,number_date,symb
0,2020-03-17,51.05,52.00,45.14,48.40,47.90,1011321.0,Puma,21,PUM.DE
1,2020-03-16,44.46,51.25,42.72,49.80,49.29,1299560.0,Puma,21,PUM.DE
2,2020-03-13,49.26,52.70,47.04,48.50,48.00,1186659.0,Puma,21,PUM.DE
3,2020-03-12,52.20,52.70,47.80,48.14,47.65,1066277.0,Puma,21,PUM.DE
4,2020-03-11,58.00,58.10,55.15,56.00,55.43,1432349.0,Puma,21,PUM.DE


In [8]:
d = datetime.strptime("Mar 17, 2020","%b %d, %Y").strftime("%Y-%m-%d")

In [9]:
d

'2020-03-17'

# for temporelle analysis

In [7]:
def scrap_data():
    data_ddf=pd.DataFrame({
        "Ets": ["Addidas","Calvin Klein"],
        "symb": ["ADDYY","PVH"]
        
        
    })
    data_ddf["l1"]="https://finance.yahoo.com/quote/"
    #Entre le 18/07 et le 16/08
    data_ddf["l2"]="/history?period1=1595030400&period2=1597536000&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true"
    data_ddf["link"]= data_ddf["l1"].str.cat(data_ddf.loc[:,["symb","l2"]])
    data_ddf["Ets"]=data_ddf["Ets"].str.strip().str.capitalize()
    print(data_ddf)
    data = []
    data_div=[] #For dividends

    for _,row in pd.DataFrame.iterrows(data_ddf):
        #print(row.Ets,row.link)
        link = row.link
        headers = {
            "User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:97.0) Gecko/20100101 Firefox/97.0"
        }
        response = req.get(link,headers=headers)
        soup=BeautifulSoup(response.content,"html.parser")
        tr_list = soup.find("tbody").find_all("tr")
        print(row.symb,len(tr_list))
        if len(tr_list) !=0:
            for line in tr_list:
                all_span = line.find_all("span")
                if all_span[-1].get_text()=="Dividend" :
                    data_div.append([all_span[0].text,line.find("strong").text,row.Ets])
                else:
                    list_values = list(map(lambda x: datetime.strptime(x.get_text(),"%b %d, %Y").strftime("%Y-%m-%d")  if all_span.index(x)==0 else  float(x.get_text().replace(",",""))  ,all_span))
                    if len(list_values)!=7 :
                        list_values.append(np.nan)
                        print(list_values)
                    list_values.append(row.Ets)
                    list_values.append(len(tr_list))
                    list_values.append(row.symb)
                    data.append(list_values)
    return data,data_div

In [8]:
data,data_div = scrap_data()
bourse_ddf = pd.DataFrame(data,columns=["date","open","high","low","close","adj_close","volume","ets","number_date","symb"])
bourse_ddf.to_csv("temp.csv",index=False)
pd.DataFrame(data_div,columns=["date","dividend","ets"]).to_csv("temp.csv_div"+".csv")

            Ets   symb                                l1  \
0       Addidas  ADDYY  https://finance.yahoo.com/quote/   
1  Calvin klein    PVH  https://finance.yahoo.com/quote/   

                                                  l2  \
0  /history?period1=1595030400&period2=1597536000...   
1  /history?period1=1595030400&period2=1597536000...   

                                                link  
0  https://finance.yahoo.com/quote/ADDYY/history?...  
1  https://finance.yahoo.com/quote/PVH/history?pe...  
ADDYY 20
PVH 20
